In [28]:
import numpy as np
import slideio
import cv2
import json
import matplotlib.path as mpltPath
import os
import warnings
import tifffile
from skimage import exposure
from skimage.exposure import match_histograms
import xml.etree.ElementTree as ET
import pandas as pd
from PIL import Image
from math import sqrt
import matplotlib.pyplot as plt
from glob import glob
from copy import copy
from tqdm.notebook import tqdm
import warnings
import staintools

# 경고 메시지를 무시하도록 설정
warnings.filterwarnings("ignore")

In [29]:
MMRstatus_data=pd.read_csv('../../data/gcu/P53&MMR.csv',encoding='cp949')
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")    
      
def svsToTiff(imageCount):
    global svsWidth,svsHeight,path,scene,fileName
    slide_tile_size=4096
    for widthCount in range(0, int(svsWidth // slide_tile_size)):
        for heightCount in range(0, int(svsHeight // slide_tile_size)):
            points = np.array([path.contains_point(np.array([widthCount* slide_tile_size, heightCount* slide_tile_size])), path.contains_point(np.array([widthCount* slide_tile_size, (heightCount+1)* slide_tile_size])), 
                                 path.contains_point(np.array([(widthCount+1)* slide_tile_size, heightCount* slide_tile_size])),
                                path.contains_point(np.array([(widthCount+1)* slide_tile_size, (heightCount+1)* slide_tile_size]))])
            inside = np.mean(points)==1
            Full=np.mean(points)>=0
            if (inside):
                image = scene.read_block((widthCount * slide_tile_size, heightCount * slide_tile_size, slide_tile_size, slide_tile_size),size=(512,512))
                if np.where(image[:,:,1]>=230)[0].shape[0]<(512*512)/2:
                    TiffImageSave(image, fileName, widthCount,heightCount)
                    imageCount += 1
            
    return imageCount
def TiffImageSave(image1,fileName,widthCount,heightCount):
    global normalizer2x,normalizer5x,normalizer10x,tmp_img
    img=Image.fromarray(image1) 
    createDirectory('../../data/gcu/tile1/'+fileName)
    img.save('../../data/gcu/tile1/'+fileName+'/'+str(widthCount*4096)+'_'+str(heightCount*4096)+'.jpg')

    

In [30]:
svs_files=glob('../../../../../../gcubme4/2022_병리 MMR Status 예측 연구_김지섭/1. 데이터/2. 작업 데이터/림프절 분리/svs/*.svs')
xml_files = [f.replace('/svs', '/xml') for f in svs_files]
xml_files = [f.replace('.svs', '.xml') for f in xml_files ]

In [31]:
for i in tqdm(range(len(svs_files))):
   
    file_extension_index=svs_files[i].find('.svs')
    fileName=os.path.basename(os.path.splitext(svs_files[i])[0])
    slide = slideio.open_slide(svs_files[i], "SVS")
    scene = slide.get_scene(0)
    svsWidth = scene.rect[2]
    svsHeight = scene.rect[3]
    imageCount=0
    magnification=80
    tile_size=256
    xml_open=xml_files[i]
    polygon_xml = ET.parse(xml_open)
    root = polygon_xml.getroot()
    root=list(root)[0]
    root=list(root)
    imageCount=0
    for annotation_count in range(len(root)):
        root1 =list(root[annotation_count])[0]
        polygon=[]
        for polygon_count in root1.iter('Coordinate'): 
            polygon.append([int(float(polygon_count.attrib['X'])),int(float(polygon_count.attrib['Y']))])
        path = mpltPath.Path(polygon)
        imageCount=svsToTiff(imageCount)


  0%|          | 0/1168 [00:00<?, ?it/s]

JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1].
JPEGFixupTagsSubsamplingSec: Warning, Auto-corrected former TIFF subsampling value

In [ ]:
svs_files=glob('../../data/OriginalData/svs/*.svs')
svs_files=['../../data/OriginalData/svs/S20-08097-8.svs']
xml_files = [f.replace('/svs', '/xml') for f in svs_files]
xml_files = [f.replace('.svs', '.xml') for f in xml_files ]

In [ ]:
tile_weight=glob('../../data/figure/tile/*.jpg')


In [ ]:
for i in tqdm(range(len(svs_files))):
   
    file_extension_index=svs_files[i].find('.svs')
    fileName=os.path.basename(os.path.splitext(svs_files[i])[0])
    slide = slideio.open_slide(svs_files[i], "SVS")
    scene = slide.get_scene(0)
    svsWidth = scene.rect[2]
    svsHeight = scene.rect[3]
    imageCount=0
    magnification=80
    tile_size=256
    xml_open=xml_files[i]
    polygon_xml = ET.parse(xml_open)
    root = polygon_xml.getroot()
    root=list(root)[0]
    root=list(root)
    imageCount=0
    slide_tile_size=4096
    Thumb=np.zeros((int(svsHeight // slide_tile_size)*50,int(svsWidth // slide_tile_size)*50,3),dtype=np.uint8)
    heat_map=np.zeros((int(svsHeight // slide_tile_size)*50,int(svsWidth // slide_tile_size)*50,3),dtype=np.uint8)
    for widthCount in range(0, int(svsWidth // slide_tile_size)):
        for heightCount in range(0, int(svsHeight // slide_tile_size)):
            image = scene.read_block((widthCount * slide_tile_size, heightCount * slide_tile_size, slide_tile_size, slide_tile_size),size=(50,50))
            matching = [s for s in tile_weight if str(widthCount*4096)+'_'+str(heightCount*4096) in s] 
            if len(matching)!=0:
                score=float(matching[0][-9:-4])
            else:
                score=0
            Thumb[heightCount*50:heightCount*50+50,widthCount*50:widthCount*50+50]=image
            image=np.zeros((50,50,3))
            image[:,:,2]=255*(1-score)
            image[:,:,0]=255*score
            heat_map[heightCount*50:heightCount*50+50,widthCount*50:widthCount*50+50]=image

In [ ]:
img=Image.fromarray(Thumb) 
img.save('../../data/figure/tile/Thumnail.jpg')
img=Image.fromarray(heat_map) 
img.save('../../data/figure/tile/heatmap.jpg')